### Performance Metrics

## Success Indicators

You have now been asked to discover what key performance indicators (KPIs) will determine the success of the new design? Use at least completion rate, time spent on each step and error rates. Add any KPIs you might find relevant.

- **Completion Rate:** The proportion of users who reach the final ‘confirm’ step.
- **Time Spent on Each Step:** The average duration users spend on each step.
- **Error Rates:** If there’s a step where users go back to a previous step, it may indicate confusion or an error. You should consider moving from a later step to an earlier one as an error.

## Redesign Outcome

Based on the chosen KPIs, how does the new design’s performance compare to the old one?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
# import reusable functions from utils directory
import sys
sys.path.append('../../utils')
import functions
import kpi_functions

In [ ]:
df = pd.read_csv('../../data/clean/final_clean_client_df.csv')

In [ ]:
df['date_time'] = pd.to_datetime(df['date_time'], errors='coerce')

# check if any dates couldn't be converted (i.e., they are NaT)
inconsistent_dates = df['date_time'].isna().sum()

# if inconsistent_dates > 0, then there are invalid or mismatched date formats
if inconsistent_dates > 0:
    print(f'There are {inconsistent_dates} inconsistent or invalid date formats in the column.')
else:
    print('All dates in the column have the same format.')

In [ ]:
kpi_df = df.copy()
kpi_df = kpi_df.drop(columns=['Unnamed: 0'])


# **Completion Rate**

In [ ]:
def find_completion_rate(df):    
    clients_finished = df[df['step'] == 4]
    total_unique_clients = df['client_id'].nunique()
    unique_clients_finished = clients_finished['client_id'].nunique()
    print(f'Clients who finished the process: {unique_clients_finished} out of {total_unique_clients}.')
    
    completion_count = clients_finished.groupby('client_id').size().reset_index(name='completion_count')
    print("Completion count (grouped by client_id):")
    print(completion_count.head())
    
    df = df.merge(completion_count[['client_id', 'completion_count']], on='client_id', how='left') # add completion_count column

    df['completion_count'] = df['completion_count'].fillna(0).astype(int) # replace NaN with 0 for clients who never finished
    
    print("Final DataFrame with completion count:")
    print(df[['client_id', 'completion_count']].head())
    
    return df

kpi_df = find_completion_rate(kpi_df)

In [ ]:
clients_finished = kpi_df[kpi_df['step'] == 4]['client_id'].nunique()
total_clients = kpi_df['client_id'].nunique()
clients_not_finished = total_clients - clients_finished

completion_rate = (clients_finished / total_clients) * 100

# pie chart
plt.figure(figsize=(8, 8))
plt.pie(
    [clients_finished, clients_not_finished],
    labels=['Finished', 'Not Finished'],
    autopct='%1.1f%%',
    colors=['lightgreen', 'salmon'],
    startangle=140
)
plt.title('Completion Rate')
plt.show()

# bar chart
plt.figure(figsize=(8, 8))
bars = plt.bar(['Total Clients', 'Completed Clients'], [total_clients, clients_finished], color=['lightgreen', 'salmon'])

# add percentage annotation on top
plt.text(1, clients_finished + 1, f'{completion_rate:.1f}%', ha='center', va='bottom', fontsize=12, color='black')

plt.title('Total Clients vs Completed Clients')
plt.ylabel('Number of Clients')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# **Time spent on each step**

New approach: calculate the time difference between consecutive steps for each client during each visit (this time taking `'visit_id'`):

- Sort the Data: Sort by `'client_id'`, `'visit_id'`, and `'step'` to ensure that steps are ordered for each client within each session.
- Calculate Time Spent on Each Step: For each `'client_id'` and `'visit_id'`, calculate the difference between the timestamp of the current step and the next step for that client.
- Group by `'client_id'` and `'step'`: Calculate the average time spent on each step.

In [ ]:
def calculate_average_time_per_step(df):
    df['date_time'] = pd.to_datetime(df['date_time'])
    
    df = df.sort_values(by=['client_id', 'visit_id', 'step'])
    
    df['next_step_time'] = df.groupby(['client_id', 'visit_id'])['date_time'].shift(-1) # calculate by shifting the 'date_time' column within each client and visit
    df['time_spent'] = df['next_step_time'] - df['date_time']
    
    step_4_rows = df[df['step'] == 4].copy()    
    step_4_rows['next_step_time'] = step_4_rows.groupby('visit_id')['date_time'].transform('max') # calculate 'next_step_time' as the max 'date_time' in each visit (end of the session)
    
    df.loc[df['step'] == 4, 'next_step_time'] = step_4_rows['next_step_time']
    df['time_spent'] = df['next_step_time'] - df['date_time'] # recalculate for step 4 after updating column
    df = df.dropna(subset=['time_spent']) # drop empty rows
    
    avg_time_per_step = df.groupby(['client_id', 'step'])['time_spent'].mean().reset_index(name='avg_time_spent') # group by and calculate
    
    # convert 'avg_time_spent' to total seconds and round up to the nearest integer using np.ceil()
    avg_time_per_step['avg_time_seconds'] = np.ceil(avg_time_per_step['avg_time_spent'].dt.total_seconds()).astype(int)
    avg_time_per_step['avg_time_minutes'] = avg_time_per_step['avg_time_seconds'] / 60

    avg_time_per_step['avg_time_minutes'] = avg_time_per_step['avg_time_minutes'].round(2) # round decimal in minutes
    
    return avg_time_per_step


time_per_step_df = calculate_average_time_per_step(kpi_df)

# print(time_per_step_df.head())

In [ ]:
step_name_mapping = {0: 'Start', 1: 'Step 1', 2: 'Step 2', 3: 'Step 3', 4: 'Finish'}
time_per_step_df['step_name'] = time_per_step_df['step'].map(step_name_mapping)

In [ ]:
# plot saved

# Bar plot for average time spent per step (without hue)
# plt.figure(figsize=(10, 6))
# sns.barplot(x='step_name', y='avg_time_minutes', data=time_per_step_df, palette='viridis')

# plt.title('Average Time Spent per Step', fontsize=16)
# plt.xlabel('Step', fontsize=12)
# plt.ylabel('Time (minutes)', fontsize=12)
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()

In [ ]:
# plot saved
# Line plot for time spent across steps (can be useful to see trends)
# plt.figure(figsize=(10, 6))
# sns.lineplot(x='step_name', y='avg_time_minutes', data=time_per_step_df, marker='o', color='blue')
# plt.title('Average Time Spent per Step', fontsize=16)
# plt.xlabel('Step', fontsize=12)
# plt.ylabel('Time (minutes)', fontsize=12)
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()

In [ ]:
# plot saved
# Box plot to show distribution of time spent per step
# plt.figure(figsize=(10, 6))
# sns.boxplot(x='step_name', y='avg_time_minutes', data=time_per_step_df)
# plt.title('Distribution of Time Spent per Step', fontsize=16)
# plt.xlabel('Step', fontsize=12)
# plt.ylabel('Time (minutes)', fontsize=12)
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()

# **Error Rates**
_(Constanza's code, taking `visit_id` into account)_

## Error count (Taking `visit_id` into account)

In [ ]:
kpi_df = kpi_df.drop(columns=['stepped_back', 'error_count'])

In [ ]:
# def calculate_error_count(df):
#     df = df.sort_values(by=['client_id', 'visit_id', 'date_time'])
#     df['stepped_back'] = df.groupby(['client_id', 'visit_id'])['step'].diff() < 0 # backward steps (errors) for all clients within each visit
#     error_counts = df.groupby('client_id')['stepped_back'].sum() # total number of errors per client
#     df['error_count'] = df['client_id'].map(error_counts).fillna(0).astype(int)
#     clients_with_errors = (df['error_count'] > 0).sum()
#     total_errors = df['stepped_back'].sum()
#     error_rate_clients = clients_with_errors / df['client_id'].nunique() * 100  # percentage of clients with errors
#     print(f'Error rate (clients with errors): {error_rate_clients:.2f}%')
#     error_rate_steps = total_errors / len(df) * 100 # percentage of total steps that are errors
#     print(f'Error rate (steps with errors): {error_rate_steps:.2f}%')
#     return df

# kpi_df = calculate_error_count(kpi_df)
# print(kpi_df.head())

In [ ]:
def calculate_error_count(df):
    df = df.sort_values(by=['client_id', 'visit_id', 'date_time'])

    # backward steps (errors) for each client within each visit
    df['stepped_back'] = df.groupby(['client_id', 'visit_id'])['step'].diff() < 0

    # Calculate total errors (total backward steps)
    total_errors = df['stepped_back'].sum()

    # total steps (rows in the dataframe)
    total_steps = len(df)

    # average error count per step (percentage of steps that are errors)
    avg_error_count_per_step = df['stepped_back'].mean()  # This gives the average proportion of steps that are errors
    print(f'Average error count per step: {avg_error_count_per_step:.2f}')

    # total error count per client (sum of backward steps per client)
    total_errors_per_client = df.groupby('client_id')['stepped_back'].sum()
    print(f'Total error count per client:\n{total_errors_per_client.head()}')

    # unique clients with errors
    clients_with_errors = (total_errors_per_client > 0).sum()
    error_rate_clients = (clients_with_errors / df['client_id'].nunique()) * 100
    print(f'Error rate (clients with errors): {error_rate_clients:.2f}%')

    # percentage of total steps that have errors
    error_rate_steps = (total_errors / total_steps) * 100  # % of steps that are errors
    print(f'Error rate (steps with errors): {error_rate_steps:.2f}%')

    # error rate per step (the proportion of steps that are errors)
    df['error_rate_per_step'] = df['stepped_back'].astype(int) / 1  # each row represents a step, so this is either 0 or 1
    error_rate_per_step = df['error_rate_per_step'].mean() * 100  # mean error rate per step
    print(f'Error rate per step: {error_rate_per_step:.2f}%')

    return df, total_errors_per_client

error_count_df, total_errors_per_client = calculate_error_count(kpi_df)
print(error_count_df.head())


In [ ]:
def plot_error_metrics(df, total_errors_per_client):
    # 1. Plot: Average error count per step
    # plt.figure(figsize=(8, 6))
    # avg_error_count_per_step = df['stepped_back'].mean() * 100  # In percentage
    # plt.barh(['Average Error Count per Step'], [avg_error_count_per_step], color='skyblue')
    # plt.xlabel('Error Rate (%)')
    # plt.title('Average Error Count per Step')
    # plt.xlim(0, 100)
    # plt.show()

    # 2. Plot: Total Error Count per Client
    plt.figure(figsize=(10, 6))
    total_errors_per_client_sorted = total_errors_per_client.sort_values(ascending=False)
    total_errors_per_client_sorted.plot(kind='bar', color='lightcoral', legend=False)
    plt.title('Total Error Count per Client')
    plt.xlabel('Client ID')
    plt.ylabel('Total Errors')
    plt.xticks(rotation=90)
    plt.show()

    # 3. Plot: Error Rate (Clients with Errors) as a percentage
    clients_with_errors = (total_errors_per_client > 0).sum()  # Clients with at least 1 error
    total_clients = df['client_id'].nunique()
    error_rate_clients = (clients_with_errors / total_clients) * 100
    plt.figure(figsize=(8, 6))
    plt.pie([error_rate_clients, 100 - error_rate_clients], labels=['With Errors', 'No Errors'], 
            colors=['skyblue', 'lightgray'], autopct='%1.1f%%', startangle=90)
    plt.title('Error Rate (Clients with Errors)')
    plt.show()

    # 4. Plot: Error Rate by Steps (Proportion of steps that are errors)
    total_steps = len(df)
    total_errors = df['stepped_back'].sum()
    error_rate_steps = (total_errors / total_steps) * 100
    plt.figure(figsize=(8, 6))
    plt.barh(['Error Rate (Steps with Errors)'], [error_rate_steps], color='lightgreen')
    plt.xlabel('Error Rate (%)')
    plt.title('Error Rate (Steps with Errors)')
    plt.xlim(0, 100)
    plt.show()

    # 5. Plot: Error Rate per Step (Proportion of errors per step)
    error_rate_per_step = df['stepped_back'].mean() * 100  # Mean error rate per step
    plt.figure(figsize=(8, 6))
    plt.barh(['Error Rate per Step'], [error_rate_per_step], color='lightpink')
    plt.xlabel('Error Rate (%)')
    plt.title('Error Rate per Step')
    plt.xlim(0, 100)
    plt.show()

# Example usage: After calculating error counts
plot_error_metrics(error_count_df, total_errors_per_client)

# **Redesign Outcome**
Based on the chosen KPIs, how does the new design’s performance compare to the old one?

For the project requirements and instructions for today’s tasks in full, please refer to the project brief. However, in order to keep on track you may refer to the daily goals outlined below:

By the end of day, we recommend you have:

Reviewed KPI and Metrics material.
Discovered what key performance indicators (KPIs) will determine the success of the new design
Use at least completion rate, time spent on each step and error rates. Add any KPIs you might find relevant.
Evaluated how the new design’s performance compare to the old one, given the chosen KPIs (completion rate, time spent on each step and error rates).

### Evaluate how the new design’s performance compare to the old one, given the chosen KPIs 
- Completion rate
- Time spent on each step
- Error rates

In [ ]:
# split in dfs for each variation
test_variation_df = kpi_df[kpi_df['variation'] == 'Test']
control_variation_df = kpi_df[kpi_df['variation'] == 'Control']
unknown_variation_df = kpi_df[kpi_df['variation'] == 'Unknown']

In [ ]:
# drop empty rows
test_variation_df = test_variation_df.dropna()
control_variation_df = control_variation_df.dropna()
unknown_variation_df = unknown_variation_df.dropna()

Convert to `.csv`

In [ ]:
# kpi_df.to_csv('kpi_df_clean.csv')

In [ ]:
# test_variation_df.to_csv('test_variation_df_clean.csv')
# control_variation_df.to_csv('control_variation_df.csv')
# unknown_variation_df.to_csv('unknown_variation_df_clean.csv')

## Completion Rates

In [ ]:
print('\nCompletion rate for Control:')
kpi_functions.find_completion_rate(control_variation_df)

In [ ]:
print('\nCompletion rate for Test:')
kpi_functions.find_completion_rate(test_variation_df)

## Time spent on each step

In [ ]:
kpi_functions.find_completion_rate(test_variation_df)

In [ ]:
kpi_functions.find_completion_rate(control_variation_df)

## Error Rate

In [ ]:
kpi_functions.find_error_rate(test_variation_df)

In [ ]:
kpi_functions.find_error_rate(control_variation_df)

In [ ]:
kpi_functions.find_error_rate(unknown_variation_df)

In [ ]:
kpi_df

# Storing values

In [ ]:
# store the results for test df
test_unique_finished, test_total_clients, test_completion_rate = kpi_functions.find_completion_rate(test_variation_df)
test_avg_time_per_step = kpi_functions.find_completion_rate(test_variation_df)
test_clients_with_errors, test_errors_per_client, test_total_errors = kpi_functions.find_error_rate(test_variation_df)

# store the results for control df
control_unique_finished, control_total_clients, control_completion_rate = kpi_functions.find_completion_rate(control_variation_df)
control_avg_time_per_step = kpi_functions.find_completion_rate(control_variation_df)
control_clients_with_errors, control_errors_per_client, control_total_errors = kpi_functions.find_error_rate(control_variation_df)

# store the results for unknown df
unknown_finished, unknown_total_clients, unknown_completion_rate = kpi_functions.find_completion_rate(unknown_variation_df)
unknown_avg_time_per_step = kpi_functions.find_completion_rate(unknown_variation_df)
unknown_clients_with_errors, unknown_errors_per_client, unknown_total_errors = kpi_functions.find_error_rate(unknown_variation_df)

## **Testing Plots**

In [ ]:
def plot_completion_rate(test_completion_rate, control_completion_rate):
    completion_rate_data = {
        'Version': ['Test', 'Control'],
        'Completion Rate': [test_completion_rate, control_completion_rate]
    }
    completion_rate_df = pd.DataFrame(completion_rate_data)
    
    plt.figure(figsize=(8, 6))
    ax = sns.barplot(x='Version', y='Completion Rate', data=completion_rate_df)
    
    ax.patches[0].set_facecolor('lightblue')
    ax.patches[1].set_facecolor('salmon')

    plt.title('Completion Rate (Test vs Control)', fontsize=16)
    plt.ylabel('Rate (%)')
    
    plt.show()

In [ ]:
# plot saved
# plot_completion_rate(test_completion_rate, control_completion_rate)

In [ ]:
def plot_error_rate(test_clients_with_errors, control_clients_with_errors, test_total_errors, control_total_errors):
    clients_error_data = {
        'Variation': ['Test', 'Control'],
        'Clients': [test_clients_with_errors, control_clients_with_errors]
    }
    clients_error_df = pd.DataFrame(clients_error_data)
    
    plt.figure(figsize=(8, 6))
    ax = sns.barplot(x='Variation', y='Clients', data=clients_error_df)
    
    ax.patches[0].set_facecolor('cadetblue')
    ax.patches[1].set_facecolor('lightcoral')    
    
    plt.title('Error Count per Client (Test vs Control)', fontsize=16)
    plt.show()

    total_errors_data = {
        'Variation': ['Test', 'Control'],
        'Total Errors': [test_total_errors, control_total_errors]
    }
    total_errors_df = pd.DataFrame(total_errors_data)
    
    plt.figure(figsize=(8, 6))
    ax = sns.barplot(x='Variation', y='Total Errors', data=total_errors_df)
    
    ax.patches[0].set_facecolor('cadetblue')
    ax.patches[1].set_facecolor('lightcoral')    
    
    plt.title('Error Count per Variation (Test vs Control)', fontsize=16)
    plt.show()

In [ ]:
# plots saved
# plot_error_rate(test_clients_with_errors, control_clients_with_errors, test_total_errors, control_total_errors)

In [ ]:
def plot_avg_time_per_step(test_avg_time_per_step, control_avg_time_per_step):
    test_avg_time_seconds = test_avg_time_per_step.dt.total_seconds().round() # convert to seconds
    control_avg_time_seconds = control_avg_time_per_step.dt.total_seconds().round()

    time_data = {
        'Process Step': ['Start', 'Step 1', 'Step 2', 'Step 3', 'Finish'],
        'Test': test_avg_time_seconds.values,
        'Control': control_avg_time_seconds.values
    }
    time_df = pd.DataFrame(time_data)
    
    time_df.set_index('Process Step').plot(kind='bar', figsize=(10, 6), width=0.8, color=['lightblue', 'salmon'])
    plt.title('Average time spent on each step (Test vs Control)', fontsize=16)
    plt.ylabel('Time (in seconds)')
    plt.xlabel('Step')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

In [ ]:
# plot_avg_time_per_step(test_avg_time_per_step, control_avg_time_per_step)

In [ ]:
# **Completion Rate Comparison - Pie Chart**

def plot_completion_rate_pie(test_completion_rate, control_completion_rate, unknown_completion_rate):
    completion_rate_data = {
        'Version': ['Test', 'Control', 'Unknown'],
        'Completion Rate': [test_completion_rate, control_completion_rate, unknown_completion_rate]
    }
    completion_rate_df = pd.DataFrame(completion_rate_data)
    
    # Pie chart
    plt.figure(figsize=(8, 8))
    plt.pie(completion_rate_df['Completion Rate'], labels=completion_rate_df['Version'], autopct='%1.1f%%', startangle=90, colors=['lightblue', 'salmon', 'lightgray'])
    plt.title('Completion Rate (Test vs Control)')
    plt.show()

In [ ]:
# plot saved
# plot_completion_rate_pie(test_completion_rate, control_completion_rate, unknown_completion_rate)

In [ ]:
def generate_error_rate_df(df):
    # Calculate total errors for each client
    total_errors_per_client = df[df['stepped_back']].groupby('client_id').size()

    # Calculate total actions per client (if each row represents an action)
    total_actions_per_client = df.groupby('client_id').size()

    # Calculate error rate as the ratio of errors to total actions
    error_rate = total_errors_per_client / total_actions_per_client

    # Create a DataFrame with all client statistics
    error_rate_df = pd.DataFrame({
        'client_id': total_actions_per_client.index,
        'total_errors': total_errors_per_client,
        'total_actions': total_actions_per_client,
        'error_rate': error_rate
    })

    # Fill missing values for clients with no errors (NaNs in total_errors and error_rate)
    error_rate_df['total_errors'] = error_rate_df['total_errors'].fillna(0).astype(int)
    error_rate_df['error_rate'] = error_rate_df['error_rate'].fillna(0).round(2)

    # Calculate error percentage (error rate * 100)
    error_rate_df['error_percentage'] = error_rate_df['error_rate'] * 100

    # Add a column for whether the client has any errors
    error_rate_df['has_error'] = error_rate_df['total_errors'] > 0

    # Optional: Add more metrics like error frequency, etc.
    # Example: error frequency based on specific time periods if there's a 'timestamp' column
    if 'timestamp' in df.columns:
        df['hour'] = pd.to_datetime(df['timestamp']).dt.hour
        error_frequency = df[df['stepped_back']].groupby(['client_id', 'hour']).size().unstack(fill_value=0)
        error_rate_df['avg_hourly_errors'] = error_frequency.mean(axis=1)

    return error_rate_df


error_rate_df = generate_error_rate_df(kpi_df)

# To view the results
print(error_rate_df.head())


In [ ]:
# plot saved

# Pie chart of Clients with Errors vs. Clients Without Errors
# clients_with_errors = error_rate_df[error_rate_df['has_error']]['client_id'].nunique()
# clients_without_errors = error_rate_df[~error_rate_df['has_error']]['client_id'].nunique()

# labels = ['Clients with Errors', 'Clients without Errors']
# sizes = [clients_with_errors, clients_without_errors]
# colors = ['#ff9999','#66b3ff']
# plt.figure(figsize=(8, 8))
# plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=colors, explode=(0.1, 0))
# plt.title('Clients with and without Errors')
# plt.show()
